# 05a3 – Compare 3SFCA Accessibility Across CT Supply Scenarios

This notebook visualises and compares spatial accessibility to CT scanners under four different supply scenarios using the Enhanced 3-Step Floating Catchment Area (E3SFCA) method.

## Objectives
- Compute E3SFCA accessibility scores under:
  - Baseline (existing 36 CT sites)
  - +5% uplift (4 new sites)
  - +10% uplift (6 new sites)
  - +20% uplift (8 new sites)
- Use consistent travel time matrix and decay weighting
- Produce 4 side-by-side maps for comparison
- Interpret differences in access coverage and equity impact


In [14]:
# ----------------------------------------------------------
# Step 0 – Imports and Display Setup
# ----------------------------------------------------------

import os
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import contextily as c
import seaborn as sns
import mapclassify
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# ----------------------------------------------------------
# Step 1 – Define File Paths and Load Baseline Inputs
# ----------------------------------------------------------

# Base directory structure
base_dir = "/Users/rosstaylor/Downloads/Research Project/Code Folder/diagnostic-modality-demand/diagnostic-modality-demand"
data_dir = os.path.join(base_dir, "data")
processed_dir = os.path.join(data_dir, "processed")
e3sfca_dir = os.path.join(processed_dir, "E3SFCA")
raw_dir = os.path.join(data_dir, "raw")

# Input files 
lsoa_access_path = os.path.join(e3sfca_dir, "lsoa_e3sfca_accessibility_2024.gpkg")
travel_matrix_path = os.path.join(e3sfca_dir, "lsoa_to_ct_travel_matrix_car.csv")

ct_sites_baseline_path = os.path.join(processed_dir, "ct_sites_capability_baseline.csv")
ct_sites_5pct_path = os.path.join(processed_dir, "ct_sites_capability_plus5pct.csv")
ct_sites_10pct_path = os.path.join(processed_dir, "ct_sites_capability_plus10pct.csv")
ct_sites_20pct_path = os.path.join(processed_dir, "ct_sites_capability_plus20pct.csv")

# Load base inputs
gdf_lsoa = gpd.read_file(lsoa_access_path)
df_travel = pd.read_csv(travel_matrix_path)

print("LSOA accessibility GeoDataFrame:", gdf_lsoa.shape)
print("Travel matrix (car mode):", df_travel.shape)


In [9]:
# ----------------------------------------------------------
# Step 2 – Load CT Supply Scenarios
# ----------------------------------------------------------

# Define supply file paths
supply_paths = {
    "baseline": ct_sites_baseline_path,
    "+5%": ct_sites_5pct_path,
    "+10%": ct_sites_10pct_path,
    "+20%": ct_sites_20pct_path
}

# Load each CSV into a GeoDataFrame with lat/lon geometry
supply_scenarios = {}
for label, path in supply_paths.items():
    df = pd.read_csv(path)
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df["longitude"], df["latitude"]),
        crs="EPSG:4326"
    )
    supply_scenarios[label] = gdf
    print(f"{label}: {gdf.shape[0]} sites loaded")


baseline: 36 sites loaded
+5%: 4 sites loaded
+10%: 6 sites loaded
+20%: 8 sites loaded


In [10]:
for label, gdf in supply_scenarios.items():
    print(f"--- {label.upper()} SCENARIO ---")
    print("Columns:", gdf.columns.tolist())
    print("First 3 rows:")
    print(gdf.head(3))
    print("\n" + "-"*40 + "\n")


--- BASELINE SCENARIO ---
Columns: ['site_name', 'scanner_count', 'site_type', 'latitude', 'longitude', 'destination_name', 'geometry']
First 3 rows:
                        site_name  scanner_count site_type   latitude  \
0     Cheltenham General Hospital              2     Acute  51.892120   
1  Gloucestershire Royal Hospital              4     Acute  51.866379   
2          Musgrove Park Hospital              3     Acute  51.011574   

   longitude destination_name                   geometry  
0  -2.071883        E01022116  POINT (-2.07188 51.89212)  
1  -2.232073        E01022312  POINT (-2.23207 51.86638)  
2  -3.121693        E01029302  POINT (-3.12169 51.01157)  

----------------------------------------

--- +5% SCENARIO ---
Columns: ['site_name', 'scanner_count', 'site_type', 'latitude', 'longitude', 'geometry', 'destination_name']
First 3 rows:
         site_name  scanner_count     site_type   latitude  longitude  \
0  Scenario Site 1              1  Scenario +5%  50.306324  

In [11]:
# ----------------------------------------------------------
# Step 3 – Assign Correct CRS (BNG) to Scenarios, Reproject to WGS84 & Recompute Lat/Lon
# ----------------------------------------------------------
for label in ["+5%", "+10%", "+20%"]:
    # 1. Grab the scenario GeoDataFrame
    gdf = supply_scenarios[label].copy()
    
    # 2. It’s actually in British National Grid but un‐labelled: force EPSG:27700
    gdf = gpd.GeoDataFrame(gdf, geometry="geometry", crs="EPSG:27700", copy=True)
    
    # 3. Now reproject to WGS84
    gdf = gdf.to_crs(epsg=4326)
    
    # 4. Recompute human-readable lat/lon
    gdf["latitude"]  = gdf.geometry.y
    gdf["longitude"] = gdf.geometry.x
    
    # 5. Overwrite back
    supply_scenarios[label] = gdf
    
    # 6. Quick sanity check
    rec = gdf[["site_name","latitude","longitude"]].iloc[0].to_dict()
    print(f"{label} scenario sample coords → {rec}")


+5% scenario sample coords → {'site_name': 'Scenario Site 1', 'latitude': 49.767253800032464, 'longitude': -7.557330219609399}
+10% scenario sample coords → {'site_name': 'Scenario Site 1', 'latitude': 49.76725356877222, 'longitude': -7.557330931673617}
+20% scenario sample coords → {'site_name': 'Scenario Site 1', 'latitude': 49.76725342542688, 'longitude': -7.557331268908025}


In [12]:
# ----------------------------------------------------------
# Step 4 – Merge Baseline + Scenario Sites and Export Combined Lists
# ----------------------------------------------------------

# Make sure your baseline is in the dict
baseline_gdf = supply_scenarios["baseline"]

# Output folder
output_dir = "/Users/rosstaylor/Downloads/Research Project/Code Folder/diagnostic-modality-demand/diagnostic-modality-demand/data/processed"
os.makedirs(output_dir, exist_ok=True)

for label in ["+5%", "+10%", "+20%"]:
    # grab the scenario sites
    scenario_gdf = supply_scenarios[label]
    
    # concatenate baseline + scenario
    combined = pd.concat([baseline_gdf, scenario_gdf], ignore_index=True)
    
    # select only the export columns
    export_df = combined[[
        "site_name", "scanner_count", "site_type",
        "latitude", "longitude", "destination_name"
    ]]
    
    # build a clean filename
    fname = label.replace("+", "plus").replace("%", "pct").lower()
    path = os.path.join(output_dir, f"ct_sites_capability_{fname}_all.csv")
    
    export_df.to_csv(path, index=False)
    print(f"Exported combined baseline + {label} → {path}")


Exported combined baseline + +5% → /Users/rosstaylor/Downloads/Research Project/Code Folder/diagnostic-modality-demand/diagnostic-modality-demand/data/processed/ct_sites_capability_plus5pct_all.csv
Exported combined baseline + +10% → /Users/rosstaylor/Downloads/Research Project/Code Folder/diagnostic-modality-demand/diagnostic-modality-demand/data/processed/ct_sites_capability_plus10pct_all.csv
Exported combined baseline + +20% → /Users/rosstaylor/Downloads/Research Project/Code Folder/diagnostic-modality-demand/diagnostic-modality-demand/data/processed/ct_sites_capability_plus20pct_all.csv


In [13]:
# ----------------------------------------------------------
# Step 5 – Verify Exported CSVs (Shapes & Column Names)
# ----------------------------------------------------------

# Directory where you exported the CSVs
output_dir = (
    "/Users/rosstaylor/Downloads/Research Project/Code Folder/"
    "diagnostic-modality-demand/diagnostic-modality-demand/data/processed"
)

# Files to check
csv_files = {
    "Baseline":           "ct_sites_capability_baseline.csv",
    "+5% All Sites":      "ct_sites_capability_plus5pct_all.csv",
    "+10% All Sites":     "ct_sites_capability_plus10pct_all.csv",
    "+20% All Sites":     "ct_sites_capability_plus20pct_all.csv"
}

for label, fname in csv_files.items():
    path = os.path.join(output_dir, fname)
    df = pd.read_csv(path)
    print(f"\n--- {label} ({fname}) ---")
    print("Shape:", df.shape)
    print("Columns:", df.columns.tolist())



--- Baseline (ct_sites_capability_baseline.csv) ---
Shape: (36, 6)
Columns: ['site_name', 'scanner_count', 'site_type', 'latitude', 'longitude', 'destination_name']

--- +5% All Sites (ct_sites_capability_plus5pct_all.csv) ---
Shape: (40, 6)
Columns: ['site_name', 'scanner_count', 'site_type', 'latitude', 'longitude', 'destination_name']

--- +10% All Sites (ct_sites_capability_plus10pct_all.csv) ---
Shape: (42, 6)
Columns: ['site_name', 'scanner_count', 'site_type', 'latitude', 'longitude', 'destination_name']

--- +20% All Sites (ct_sites_capability_plus20pct_all.csv) ---
Shape: (44, 6)
Columns: ['site_name', 'scanner_count', 'site_type', 'latitude', 'longitude', 'destination_name']
